## 1.1 Feature Engineering

In [1]:
import pandas as pd
import holidays
import numpy as np
import datetime as dt

#Daten einladen

data_Preise = pd.read_excel("Daten_Preise.xlsx", sheet_name='Tabelle1', skiprows=1, index_col=0)
data_Preise = data_Preise.resample('h').ffill()
data_Preise.index = pd.to_datetime(data_Preise.index, format= "%Y-%m-%d %H:%M:%S")

data_Sonstige = pd.read_excel("Daten_sonstige_h.xlsx", sheet_name='Tabelle1', skiprows=1, index_col=0)
data_Sonstige.index = pd.to_datetime(data_Sonstige.index, format= "%d.%m.%Y %H:%M")

# Index in ein einheitliches Format übertragne
data_Preise.index = data_Preise.index.strftime("%Y-%m-%d %H:%M:%S")
index1 = data_Preise.index
data_Sonstige.index = data_Sonstige.index.strftime("%Y-%m-%d %H:%M:%S")
index2 = data_Sonstige.index

#Daten in einem DataFrame zusammenfügen
data_roh = pd.merge(data_Sonstige, data_Preise, left_index=True, right_index=True, how='inner')
data_roh.index = pd.to_datetime(data_roh.index)

df_cleaned = data_roh

KeyboardInterrupt: 

In [4]:
#Ideen für Feature Engineering

#Zeitliche Einflüsse
#Tag des Jahres
df_cleaned["Day-of-year"] = df_cleaned.index.dayofyear
#Monat
df_cleaned["Monat"] = df_cleaned.index.month
# Stunde des Tages
df_cleaned["Stunde"] = df_cleaned.index.hour
#Feiertage
de_holidays = holidays.Germany(years=range(2015, 2022))
df_cleaned['Feiertag'] = df_cleaned.index.isin(de_holidays).astype(int)


#Berücksichtigung Ausgleichsenergie spez. Kosten 
df_cleaned["Ausgleichsenergie [€]"] = (df_cleaned["Volumen (+) [MWh]"]+df_cleaned["Volumen (-) [MWh]"])*df_cleaned["Preis [€/MWh]"] #Überprüft nicht geeignet. Kein Zusammenhang mit großer Auswirkung erkennbar


#Kraftwerke mit Preisen
df_cleaned['Summe_CO2'] = df_cleaned['Braunkohle [MWh]']* 1.188 +df_cleaned['Steinkohle [MWh]']* 0.95 +df_cleaned['Erdgas [MWh]']*0.4
df_cleaned['Summe_Kohle'] = df_cleaned['Braunkohle [MWh]']*0.2398 +df_cleaned['Steinkohle [MWh]']*0.1228
df_cleaned['Kraftwerke_CO2'] = (df_cleaned['Braunkohle [MWh]']* 1.188 +df_cleaned['Steinkohle [MWh]']* 0.95 +df_cleaned['Erdgas [MWh]']*0.4)*df_cleaned['CO2 [€/t]']
df_cleaned['Kraftwerke_Kohle'] = (df_cleaned['Braunkohle [MWh]']*0.2398 +df_cleaned['Steinkohle [MWh]']*0.1228 )*df_cleaned['Kohle [€/t]']
df_cleaned['Kraftwerke_Gas'] = df_cleaned['Erdgas [MWh]']*df_cleaned['Gas [€/MWh]']
df_cleaned['Kosten_Erdgas'] = (df_cleaned['Erdgas [MWh]']* df_cleaned['Gas [€/MWh]'])+ (df_cleaned['Erdgas [MWh]']*0.4*df_cleaned['CO2 [€/t]'])
df_cleaned['Kosten_Braunkohle'] = (df_cleaned['Braunkohle [MWh]']*0.2398*df_cleaned['Kohle [€/t]'])+ (df_cleaned['Braunkohle [MWh]']*1.188*df_cleaned['CO2 [€/t]'])
df_cleaned['Kosten_Steinkohle'] = (df_cleaned['Steinkohle [MWh]']*0.1228*df_cleaned['Kohle [€/t]'])+ (df_cleaned['Steinkohle [MWh]']*0.95*df_cleaned['CO2 [€/t]'])
df_cleaned['Kraftwerke_Fossil'] = df_cleaned['Kraftwerke_Gas']*df_cleaned['Kraftwerke_Kohle']*df_cleaned['Kraftwerke_CO2']

#Quellen-CO2: 
#Braunkohle: hhttps://www.bund-nrw.de/braunkohle/hintergruende-und-publikationen/braunkohlenkraftwerke-contra-klimaschutz/
#Steinkohle: https://www.brisant.de/braunkohle-umwelt-100.html
#Erdgas: https://www.ise.fraunhofer.de/de/presse-und-medien/news/2019/33-prozent-weniger-co2-emissionen-durch-brennstoffwechsel-von-kohle-auf-gas.html

#Quellen-Kohle:
#Braunkohle: https://agrarplus.at/heizwerte-aequivalente.html
#Steinkohle: https://www.unitjuggler.com/energy-umwandeln-von-MWh-nach-tSKE.html


# Anteil Erneuerbarer
df_cleaned['Summe_Ern']= df_cleaned[['Wind Offshore [MWh]', 'Wind Onshore [MWh]', 'Photovoltaik [MWh]','Biomasse [MWh]', 'Sonstige Erneuerbare [MWh]','Wasserkraft [MWh]',]].sum(axis=1)
df_cleaned['Anteil Erneuerbarer'] = df_cleaned['Summe_Ern'] / (df_cleaned['Gesamt (Netzlast) [MWh]']) * 100
df_cleaned['Anteil_PV'] = df_cleaned['Photovoltaik [MWh]']/df_cleaned['Summe_Ern']
df_cleaned['Anteil_Offshore'] = df_cleaned['Wind Offshore [MWh]']/df_cleaned['Summe_Ern']

df_cleaned['Prognose_Summe_Ern']= df_cleaned[['Prognose Wind Offshore [MWh] ', 'Prognose Wind Onshore [MWh]', 'Prognose Photovoltaik [MWh]']].sum(axis=1)
prog_proz_ern = df_cleaned['Prognose_Summe_Ern'] / (df_cleaned[' Prognose Gesamt (Netzlast) [MWh]']) * 100
df_cleaned['Prognose Anteil Erneuerbarer'] = prog_proz_ern

# Überschuss Erzeugung
summe_erzeugung = df_cleaned[['Biomasse [MWh]', 'Wasserkraft [MWh]',
       'Wind Offshore [MWh]', 'Wind Onshore [MWh]', 'Photovoltaik [MWh]',
       'Sonstige Erneuerbare [MWh]', 'Kernenergie [MWh]', 'Braunkohle [MWh]',
       'Steinkohle [MWh]', 'Erdgas [MWh]', 'Erzeugung Pumpspeicher [MWh]',
       'Sonstige Konventionelle [MWh]']].sum(axis=1)
df_cleaned['Überschuss'] = summe_erzeugung - df_cleaned['Gesamt (Netzlast) [MWh]']


# Abweichung Prognose Residuallast
df_cleaned['Abweichung_Prognose_Residuallast'] = (df_cleaned['Residuallast [MWh]'] - df_cleaned['Prognose Residuallast [MWh]'])


#mögliche Interaktionen zwischen Features
df_cleaned['Int_Gas'] = df_cleaned['Gas [€/MWh]']*df_cleaned['Gesamt (Netzlast) [MWh]']
df_cleaned['Int_Gas_Resi'] = df_cleaned['Gas [€/MWh]']*df_cleaned['Residuallast [MWh]']
df_cleaned['Int_CO2_Erz'] = summe_erzeugung*df_cleaned['CO2 [€/t]']
df_cleaned['Int_CO2_Kohle'] = df_cleaned['Braunkohle [MWh]']* 1.188 +df_cleaned['Steinkohle [MWh]']* 0.95*df_cleaned['CO2 [€/t]']
df_cleaned['Int_Ern_Foss'] = df_cleaned['Anteil Erneuerbarer'] * df_cleaned['Erdgas [MWh]']*df_cleaned['Steinkohle [MWh]']*df_cleaned['Braunkohle [MWh]']
df_cleaned['Int_Preise'] = df_cleaned['Kraftwerke_CO2']*df_cleaned['Kraftwerke_Kohle']*df_cleaned['Kraftwerke_Gas']
df_cleaned['Int_Pumpspeicher'] = df_cleaned['Pumpspeicher [MWh]']/df_cleaned['Kraftwerke_Fossil']

# Versuch Einbau der Merit-Order als Feature
merit_order = ['Wind Offshore [MWh]', 'Wind Onshore [MWh]', 'Photovoltaik [MWh]', 'Wasserkraft [MWh]','Biomasse [MWh]',
               'Sonstige Erneuerbare [MWh]', 'Kernenergie [MWh]', 'Braunkohle [MWh]', 'Steinkohle [MWh]', 'Erdgas [MWh]',
               'Erzeugung Pumpspeicher [MWh]', 'Sonstige Konventionelle [MWh]']

df_order = df_cleaned[merit_order]
df_cumsum = df_order.cumsum(axis=1)

gesamtlast_serie = df_cleaned['Gesamt (Netzlast) [MWh]']

def finde_lastdeckung(row, gesamtlast):
    for kraftwerk, kum_erz in row.items():
        if kum_erz >= gesamtlast:
            return kraftwerk, kum_erz
        return None, None

#df_cleaned['Deckendes Kraftwerk'], df_cleaned['Erzeugung bei Deckung'] = zip(*df_cumsum.apply(lambda row: finde_lastdeckung(row, gesamtlast_serie[row.name]), axis = 1))
#print(df_cleaned[['Deckendes Kraftwerk', 'Erzeugung bei Deckung']])

#Standardabweichung
#df_cleaned['Std_Gas'] = df_cleaned['Gas [€/MWh]'].rolling(window=24).std()
#df_cleaned['Std_Kohle'] = df_cleaned['Kohle [€/t]'].rolling(window=24).std()
#df_cleaned['Std_CO2'] = df_cleaned['CO2 [€/t]'].rolling(window=24).std()
#df_cleaned['Std_Erneuerbare_Tag'] = df_cleaned['Summe_Ern'].rolling(window=24).std()
#df_cleaned['Std_Prognose'] = df_cleaned['Abweichung_Prognose_Residuallast'].rolling(window=24).std()



#Gleitender Durchschnitt Preise
#df_cleaned['Moving_Average_Gas'] = df_cleaned['Gas [€/MWh]'].rolling(window=2500).mean()
#df_cleaned['Moving_Average_Kohle'] = df_cleaned['Kohle [€/t]'].rolling(window=2500).mean()
#df_cleaned['Moving_Average_CO2'] = df_cleaned['CO2 [€/t]'].rolling(window=2500).mean()

df_cleaned

C:\Users\Dominik\AppData\Local\Temp\ipykernel_13032\2105469522.py:12: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_cleaned['Feiertag'] = df_cleaned.index.isin(de_holidays).astype(int)


,Wochentag,Prognose Wind Offshore [MWh],Prognose Wind Onshore [MWh],Prognose Photovoltaik [MWh],Prognose Sonstige [MWh],Prognose Gesamt (Netzlast) [MWh],Prognose Residuallast [MWh],Biomasse [MWh],Wasserkraft [MWh],Wind Offshore [MWh],Wind Onshore [MWh],Photovoltaik [MWh],Sonstige Erneuerbare [MWh],Kernenergie [MWh],Braunkohle [MWh],Steinkohle [MWh],Erdgas [MWh],Erzeugung Pumpspeicher [MWh],Sonstige Konventionelle [MWh],Gesamt (Netzlast) [MWh],Residuallast [MWh],Pumpspeicher [MWh],Nettoexport [MWh],Day-ahead Auktion (volumengewichtet) [€/MWh],Volumen (+) [MWh],Volumen (-) [MWh],Preis [€/MWh],Temperatur,Kohle [€/t],Gas [€/MWh],CO2 [€/t],BIP,Verbraucherpreisindex,Day-of-year,Monat,Stunde,Feiertag,Ausgleichsenergie [€],Summe_CO2,Summe_Kohle,Kraftwerke_CO2,Kraftwerke_Kohle,Kraftwerke_Gas,Kosten_Erdgas,Kosten_Braunkohle,Kosten_Steinkohle,Kraftwerke_Fossil,Summe_Ern,Anteil Erneuerbarer,Anteil_PV,Anteil_Offshore,Prognose_Summe_Ern,Prognose Anteil Erneuerbarer,Überschuss,Abweichung_Prognose_Residuallast,Int_Gas,Int_Gas_Resi,Int_CO2_Erz,Int_CO2_Kohle,Int_Ern_Foss,Int_Preise,Int_Pumpspeicher
ContractBegin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-05 00:00:00,2,551.75,10009.75,0.0,47097.50,43176.00,32614.50,4064.00,1680.25,473.75,11847.25,0.00,132.25,10934.50,16041.75,2201.50,987.00,1334.25,5207.00,46668.00,34347.00,2429.25,9906.00,17.93,0,-844,-37.56,3.400,64.7,7.24,20.354,100.0,100.0,5,1,0,0,31700.64,21543.8240,4117.15585,4.385030e+05,2.663800e+05,7145.880,15181.6392,6.367871e+05,60060.134190,8.346990e+14,18197.50,38.993529,0.000000,0.026034,10561.50,24.461506,8235.50,1732.50,337876.32,248672.28,1.117506e+06,61626.463450,1.359190e+12,8.346990e+14,2.910331e-12
2015-01-05 01:00:00,2,547.75,10287.75,0.0,44789.50,41597.75,30762.25,4069.25,1676.50,464.75,11814.75,0.00,131.00,11055.00,15390.50,2163.25,961.25,1038.25,4743.25,44627.00,32347.50,2823.00,10138.00,15.17,0,-589,-15.97,3.150,64.7,7.24,20.354,100.0,100.0,5,1,1,0,9406.33,20723.5015,3956.28900,4.218061e+05,2.559719e+05,6959.450,14785.5630,6.109353e+05,59016.618345,7.514154e+14,18156.25,40.684451,0.000000,0.025597,10835.50,26.048284,8880.75,1585.25,323099.48,234195.90,1.089097e+06,60113.164975,1.302040e+12,7.514154e+14,3.756910e-12
2015-01-05 02:00:00,2,542.50,10418.50,0.0,43457.00,40421.75,29460.75,4068.50,1661.50,461.50,12198.75,0.00,131.00,10861.00,15407.75,2012.00,983.00,265.50,5266.50,43635.50,30975.25,3629.75,10350.00,16.38,0,-625,-15.35,2.825,64.7,7.24,20.354,100.0,100.0,5,1,2,0,9593.75,20609.0070,3941.85205,4.194757e+05,2.550378e+05,7116.920,15120.1128,6.116201e+05,54890.297520,7.613836e+14,18521.25,42.445371,0.000000,0.024917,10961.00,27.116589,9681.50,1514.50,315921.02,224260.81,1.085214e+06,57209.042600,1.293454e+12,7.613836e+14,4.767308e-12
2015-01-05 03:00:00,2,533.75,10601.75,0.0,43335.50,40404.25,29268.75,4067.00,1644.00,450.25,12312.75,0.00,131.00,10760.00,15528.75,2049.25,996.75,274.00,5612.25,43683.25,30920.25,3438.00,10439.00,17.38,0,-494,-14.54,2.575,64.7,7.24,20.354,100.0,100.0,5,1,3,0,7182.76,20793.6425,3975.44215,4.232338e+05,2.572111e+05,7216.470,15331.6098,6.164232e+05,55906.531905,7.855881e+14,18605.00,42.590696,0.000000,0.024200,11135.50,27.560220,10142.75,1651.50,316266.73,223862.61,1.095574e+06,58073.067775,1.350929e+12,7.855881e+14,4.376339e-12
2015-01-05 04:00:00,2,521.75,10809.75,0.0,44479.50,41348.25,30016.75,4052.00,1632.00,453.50,13093.50,0.00,131.00,10632.75,15449.25,2783.50,1044.75,5.25,6223.75,44969.25,31422.25,3785.00,10705.00,16.38,0,-1294,-61.14,2.375,64.7,7.24,20.354,100.0,100.0,5,1,4,0,79115.16,21415.9340,4046.54395,4.358999e+05,2.618114e+05,7563.990,16069.9266,6.132674e+05,75937.943910,8.632295e+14,19362.00,43.056088,0.000000,0.023422,11331.50,27.405029,10532.00,1405.50,325577.37,227497.09,1.129672e+06,72176.300050,1.934397e+12,8.632295e+14,4.384697e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..